In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.formula.api import ols #for linear regression
import seaborn as sns #for heatmap
import statsmodels.api as sm
from scipy import stats
from sklearn.metrics import r2_score #R-squared
from sklearn.model_selection import train_test_split 
from sklearn.metrics import r2_score, mean_squared_error#for train and test split
from statsmodels.api import qqplot 
from scipy.stats import shapiro,ttest_ind
from statsmodels.stats.diagnostic import het_breuschpagan,linear_rainbow  
from sklearn import linear_model

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [ ]:
# read the data 

data = pd.read_excel(r'C:\\Users\\User\\Downloads\\Student_data.xlsx')

In [ ]:
data.head(3)

In [ ]:
#looking at the columns of the dataset

print(data.columns.tolist())
print(" ")
print("We have overall", len(data.columns), "columns")

In [ ]:
print("The shape of our data is:", data.shape)

In [ ]:
data.describe()

When looking at the data descibe, we can some descriptive information about each variable, can think of a possible distribution. 
1. StudentID does not give us any important information from this perspective, sincce it's not made for modeling purposes. 
2. Age: looking at the description of the variable, we can see that the age of the students is between 15 to 22. The average age is about 17. Also, we can see info about the major quantiles and how the variable is spread through them. 

Let's skeep the other variables for now

## Studying about and handling the missing and unique values

In [ ]:
print("Overall number of missing values is", data.isna().sum().sum())

In [ ]:
print("Overall number of duplicate values is:", data.duplicated().sum())

As we can see, we do not have nulls and can continue working with our data without deleting the rows or making changes in the values.  

In [ ]:
data.columns[data.columns.nunique==1]

Also, as we can see, no variable has 1 unique value... If we had such one, we would need to derop that 

In [ ]:
data.info()

As we can see, all of our variables are objects or integers, which makes the work easier, no need for additional work here. 

In [ ]:
data.nunique()

The above code shows the number of unique values for each variable. This helps to understand the need of modifications before dummifying the categorical variables to decrease the number of unnecessary categories. 

### Going through the categories

By using the following method, we can look through each variable and understand the existing categories

In [ ]:
print(type(data.famsize))
print(data.famsize.unique())

We can run a loop and see the unique values of each variable 

In [ ]:
for col in data:
    print("The unique value in ",col,"are:" , data[col].unique())
    print(col, ": \n", data[col].value_counts(), "\n ........")

## Understanding the Distributions

In [ ]:
# Density Plot and Histogram for the Final Grade 
plt.figure(figsize = (10,7))
plt.title("Final Grade Distribution",fontsize = 15, color = 'darkblue') 
sns.distplot(data.Final_Grade, hist=True, kde=True, 
             bins=int(180/5), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})

Without the point 0,  the distribution of Final Grade would be similar to Normal distribution. Howevfer, a lot of students got 0 and the distribution has some gaps. 

### Let's look at the outliers 
As the final grade has a specific range, most probably, we will not have any outliers 

In [ ]:
plt.figure(figsize = (10,7))
sns.boxplot(x="Final_Grade",data=data, palette='cool',hue=data.paid,  showmeans=True)
plt.legend()
plt.title('Final grade outliers',fontsize = 15, color = 'darkblue')
plt.show()

Here we can see that the 50% quantile of final grade is about 11, here we do not have outliers. Most of the values are centered on the right side, meaning, they are higher than 10 in this case. 

Let's look some other variables' distributions as well

In [ ]:
for i in ["age", "studytime", "failures", "absences"]:
    plt.figure(figsize = (10,7))
    sns.scatterplot(data[i],data["Final_Grade"])
    plt.xlabel(i)
    plt.ylabel("Final Grade")
    plt.title(f"Relationship between {i} and the Final Grade")
    plt.show()

The above cell shows scatterplots of the given variables (age, studytime, failures, absences) and final grade. 

### The following Bubble plot shows the connection between absences and final grade, where failures are mentioned

In [ ]:
plt.figure(figsize = (10,7))

sns.scatterplot(data=data, x="absences", y="Final_Grade", size="failures", 
                legend=True, sizes=(20, 1000), alpha=0.6, color = 'darkblue')
plt.legend( prop={"size":20})
# show the graph
plt.show()

As we can see, as the number of failures increases, the final rade decreases. Also, the more the absences are, the lower the grade is. 

In [ ]:
np.corrcoef(data.absences,data.Final_Grade)

In [ ]:
np.corrcoef(data.failures,data.Final_Grade)

## Separating object and numeric datatypes

In [ ]:
# separate numeric and object parts 

data_num = data.select_dtypes(exclude = ["object"])
data_obj = data.select_dtypes(include = ["object"])

In [ ]:
print("The shape of the numeric data is:", data_num.shape)
print("The shape of the object data is:", data_obj.shape)

Looking at the unique values, we can see that no varible has "unnecessary" category, which can be grouped with other variables. 

As we can see, in both mother's case and  father's case, healthcare job has the lowest number. So, let's group it with the "other" category, in order not to have so many variables after dummifying. 

## Working with categorical variables

### Let's graphically interpret the differences between the categories of object types and final grade

In [ ]:
list_obj = data_obj.columns.tolist()

In [ ]:
#for comparing Final grade with categorical variables it represented in boxplot
#We can see difference in final grades by address,Fjob teacher level,Mjob at health level,internet, schoolsup,higher.
for i in list_obj:
    sns.boxplot(data=data,x=i,y="Final_Grade", showmeans=True, palette='cool')
    plt.show()

1. School2 
As we can see, in terms of grades GP has higher average (median) grade, however, the grade range is higher. 

2. Sex
Results show that males tend to get higher grades than females 

3. Address
The students from urban areas tend to get higher scores. Their distribution is approximatelly looking like normal, and the range is smaller than that of Region students 

4. Famsize 
In both cases, the median is approximately equal 

5. Pstatus 
Apart has smaller grade range, but has an outlier 


etc. 
Let's leave the other variables, since nothing interesting can be interpreted 

## Working with numeric variables

In [ ]:
data_num.head()

Let's see which variables have high correlation which do not.  
Here, let's take 0.8 as the decision making number, meaning, if the correlation is higher or equal to 0.8 we are going to decide on keeping or deleting one of the variables. 

In [ ]:
# correlation heatmap for examining multicollinearity

plt.figure(figsize = (12,8))
sns.heatmap(data_num.corr().abs().round(2), annot =True, cmap="PuBu")

Here the variables do not have strong correlation. As we can see from the graph, none of them had >=0.8 correlation. 
Only, studentID and age have high correlation. But the variables are not related, and rationally thinking, studentID does not have any impact on the final grade. 

Walc and Dalc have the seond highest values. Logically, they can have some correlation, in a sense that a person likes drinking alcohol or not. However, the results are less than the target value, so let's leave the variables. 

Since we do not need ID variable for the modeling, let's remove that 

In [ ]:
data = data.drop("StudentID", axis = 1)

In [ ]:
data_num = data_num.drop("StudentID", axis = 1)

In [ ]:
plt.figure(figsize = (6,4))
sns.pairplot(data_num)


# Visual representation of pairwise relationships
# The plot is made to see the highest correlated variables with the Final Grade

The graph shows the linear relationship between the Final Grade and all the numeric variables.

## Modeling

Since about half of our variables are categorical (in object type), we can convert their meaning into numbers 

In [ ]:
# Let's dymmify the variables

data_dummies = pd.get_dummies(data_obj, drop_first = True)
data_dummies.head()

In [ ]:
data_dummies.shape

In [ ]:
data_model = data_num.join(data_dummies, how = 'left')
data_model.shape

In [ ]:
Y = data_model.Final_Grade
X = data_model.drop('Final_Grade', axis = 1) # we drop Final Grade in order not to have the target in the X part  
X = sm.add_constant(X) # add constant as an intercept 

Let's split our data into two parts, where 75% of our data will be used in the train part and 25% in test. 

In [ ]:
# train test split data 
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.25, random_state=17)
train_x.shape

As we can see, the number of columns has decreases by 1, which was for Final Grade, target variable 
And we took 296 values for the train test

In [ ]:
# Let's create the linear model 

model_linear = sm.OLS(train_y, train_x)
results = model_linear.fit()
results.summary()

Let's look at each point one by one: 

1. R-squared is low, meaning, our independent variable is not explaining much in the variation of your dependent variable - regardless of the variable significance
2. Adj. R-squared is even lower, which indicates that the additional input variables are not adding value to the model.
3. F-stat is close to 0, which is good
4. We will use AIC and BIC later, when we have different models and we will need to take one with higher AIC and BIC values (if R-squared is not informative) 

Now, let's consider t and p-value, from which we can understan that const is not significant  

In [ ]:
# check prediction on the test set
y_pred=results.predict(test_x)
print('R^2_test:', r2_score(test_y, y_pred))
print('RMSE:', mean_squared_error(test_y, y_pred)**0.5)

The difference between train and test values is on average 4.31 point

F-stat is too small, near to 0, which means that our model is statistically significant at 5%(even 1%) significance level, so there is at least one estimated coefficient (besides intercept) that is not null. 

In [ ]:
#R-squared shows that 33.5% of variance in Final Grades is explained by the variables included in the model.

#Adj. R-squared 24% is not the same as (or close to) R-squared, which means that not all of the included variables are important.

As we can see, the model is not strong enough. So, let's remove some of the unnecessary variables

Let's understand which variable is important and which is not, to remove them correctly and go to the estimation part again. 

First model is complete, let's continue with feature engineering

## Feature Engineering

In [ ]:
# transformm to the binary feature

In [ ]:
# Let's eliminate some unsignificant features according to p-value
p_values = results.pvalues.round(3).reset_index().rename(
    columns={
        'index':'features',
        0: 'p_value'
    }
).sort_values("p_value")


In [ ]:
# Let's eliminate features that have more than 0.5 p_value
bad_features_df = p_values[p_values.p_value > 0.5]
bad_features = bad_features_df.features.tolist()
print(bad_features)

In [ ]:
# Let's eliminate features that have more than 0.5 p_value
bad_features_df = p_values[p_values.p_value > 0.5]
bad_features = bad_features_df.features.tolist()
print(bad_features)

In [ ]:
print("Overall we have", len(bad_features), "bad features!")

In [ ]:
# As we know what features are bud let's run multiple regressions without them (eliminateing one by one)
for feature in bad_features:
    # temporary variables to save the new dataset
    temporary_train_x = train_x.drop([feature], axis = 1)
    temporary_test_x = test_x.drop([feature], axis = 1)
    # temporary models to see how it changes if bad features are eliminated
    temporary_model = sm.OLS(train_y, temporary_train_x)
    temporary_results = temporary_model.fit()
    print(f"This is a evaluation of the model without {feature.upper()} feature")
    print("Train R^2:",temporary_results.rsquared.round(3))
    print("Train R^2 Adjusted:",temporary_results.rsquared_adj.round(3))
    # check prediction on the test set
    temporary_y_pred=temporary_results.predict(temporary_test_x)
    print('R^2_test:', r2_score(test_y, temporary_y_pred).round(3))
    print('RMSE test:', (mean_squared_error(test_y, temporary_y_pred)**0.5).round(3))
    print()

As we can see, the insignificant features have so little impact on our model so eliminating them will better the model(though slightly)

In [ ]:
np.corrcoef(data_model.Dalc, data_model.Walc)

Dalc and Walc have some correlation

In [ ]:
#Dalc and Walc have similar meaning, so we can work on them 
#New variable will represent  weekly alcohol consumption
data_model_new = data_model

data_model_new["alc"]=5/7*data_model_new["Dalc"]+2/7*data_model_new["Walc"]

# we take 5 for weekdays and 2 for weekends 

In [ ]:
len(data_model_new.columns.tolist())

In [ ]:
data_model_new.columns

As we can see, a new column is added

In [ ]:
#thus we will merge freetime, traveltime and free time after school by taking mean of the two columns
data_model_new["Freetime"]=data_model_new[['freetime', 'goout', 'traveltime']].mean(axis=1)

In [ ]:
len(data_model_new.columns)

## Final model

In [ ]:
# train-test split data
Y = data_model_new.Final_Grade
X = data_model_new.drop('Final_Grade', axis = 1)
X = sm.add_constant(X)
train_engineered_x, test_engineered_x, train_engineered_y, test_engineered_y = train_test_split(
    X,
    Y,
    test_size=0.25,
    random_state=42
)
train_engineered_x.shape

In [ ]:
#splitting the data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=42)

In [ ]:
#observing the results
results2=sm.OLS(Y_train, X_train).fit()
results2.summary()

In [ ]:
# check prediction on the test set (initial)
y_pred=results.predict(test_x)
print('R^2_test:', r2_score(test_y, y_pred))
print('RMSE:', mean_squared_error(test_y, y_pred)**0.5)

In [ ]:
# check prediction on the test set
y_engineered_pred=results2.predict(test_engineered_x)
print('R^2_test:', r2_score(test_engineered_y, y_engineered_pred))
print('RMSE:', mean_squared_error(test_engineered_y, y_engineered_pred)**0.5)

As we can see, here we have big positive change in terms R-squared. 
And RMSE is decreased, which is also a good sign. 

Also, AIC and BIC have better results, since, 
<br>   initial: AIC=1718, BIC=1866
<br>   new model: AIC=1938, BIC=2090

#### Interpretation of the results of Linear Regression
F-statistics (2.27e-07) : As Prob (F-statistic) shows our model is statistically significant at 5%(even 1%) significance level, so there is at least one estimated coefficient (besides intercept) that is not null.

R-squared has low value and which shows that only 27.5% of variance in final grade is explained by the variables included in the model.

Adj. R-squared: As summary shows it's 18%. It is not quite close to R-squared, which means that there are not important variables included in the model, with which we are going to work in the next part. 

Statistically significant variables: 
  - **failures** has a **p-value** equal to zero which means that it is statsitically significant, and as the coefficient is positive it's effect on price is positive.
  
  Other variables, such as **Studytime, sexM, famsup_Yes, and romantic_Yes** are considered to be important variables. 
  - Other variables are not statistically significant at 95% conidence interval, but some are significant at 90%. 
  
Final Model

Please note that if the variable is in the model and you have no possibility or you do not want to remove it it should be in the final model formulation even if it is not statistically significant

**Final Grade = 12.4613 - 0.3100 age + 0.3915 Medu - 0.0815 Fedu + ...**

### Let's eliminate some unsignificant features according to p-value

In [ ]:
p_values = results2.pvalues.round(3).reset_index().rename(
    columns={
        'index':'features',
        0: 'p_value'
    }
).sort_values("p_value")
p_values.fillna(1,inplace=True)

In [ ]:
# Let's eliminate features that have more than 0.5 p_value
bad_features_after_engineering_df = p_values[p_values.p_value > 0.5]
bad_features_after_engineering = bad_features_after_engineering_df.features.tolist()
print(bad_features_after_engineering)

In [ ]:
# As we know what features are bud let's run multiple regressions without them (eliminateing one by one)
feature_list = list()
for feature in bad_features:
    feature_list.append(feature)
    # temporary variables to save the new dataset
    temporary_train_x = train_engineered_x.drop(feature_list, axis = 1)
    temporary_test_x = test_engineered_x.drop(feature_list, axis = 1)
    # temporary models to see how it changes if bad features are eliminated
    temporary_model = sm.OLS(train_engineered_y, temporary_train_x)
    temporary_results = temporary_model.fit()
    print(f"This is a evaluation of the model without {feature.upper()} feature")
    print("Train R^2:",temporary_results.rsquared.round(3))
    print("Train R^2 Adjusted:",temporary_results.rsquared_adj.round(3))
    # check prediction on the test set
    temporary_y_pred=temporary_results.predict(temporary_test_x)
    print('R^2_test:', r2_score(test_engineered_y, temporary_y_pred).round(3))
    print('RMSE test:', (mean_squared_error(test_engineered_y, temporary_y_pred)**0.5).round(3))
    print()

In [ ]:
feature_list

In [ ]:
Final_model = temporary_results.summary()
Final_model

In [ ]:
# we specify do not fit intercept because we already have the constant feature
reg = linear_model.LinearRegression(fit_intercept=False).fit(train_engineered_x, train_engineered_y)
print('R^2_train:', reg.score(test_engineered_x, test_engineered_y))
print('R^2_train:', reg.score(test_engineered_x, test_engineered_y))

In [ ]:
# the package do not have the same summary, and the code below only provides the features and respective coefficients
coef = pd.DataFrame(dict(zip(train_engineered_x.columns, reg.coef_)), index=[0]).T
coef.columns=['coef']
coef

## Model Diagnostics

In [ ]:
#residuals of the model
residuals1=results2.resid
predicted_values1=results2.predict()

### 1. Homoscedasticity assumptions

In [ ]:
#predicted values vs residuals
plt.figure(figsize=(10,5))
sns.scatterplot(predicted_values1,residuals1)
plt.axhline(y=0, c="red")
plt.title("Residuals vs Predicted values for backward eliminated model")
plt.ylabel("Residuals")
plt.xlabel("Predicted values")
plt.show()

In [ ]:
#Breusch-Pagan test for homoscedasticity
bnames = ['Lagrange multiplier statistic (for homoscedasticity)', 'p-value','f-value', 'f p-value']
breush = het_breuschpagan(residuals1, results2.model.exog)
print(list(zip(bnames, breush)))

#### Homoscedasticity assumptions -- fail

### 2. Normality assumption

In [ ]:
#Normality
sns.distplot(residuals1)
plt.show()

In [ ]:
#Shapiro-Wilk normality test. 
#Normality assumption isn't held.
snames=['The test statistic for Normality', 'p-value']
shapiro_test=shapiro(residuals1)
print(list(zip(snames, shapiro_test)))

#### Normality assumption -- fail

### 3. Linearity assumption

In [ ]:
#QQ plot
qqplot(residuals1,fit=True)
plt.show()

In [ ]:
#Linearity 
rnames=["fstat", "p-value"]
rainbow=linear_rainbow(results2)
print(list(zip(rnames, rainbow)))

#### Linearity assumption is held

## Let's check multicollinearity

#### VIF: measure of the amount of multicollinearity in a set of multiple regression variables.

In [ ]:
x_list=X.columns.tolist()

In [ ]:
#Multicollinearity
#calculating vif using variance_inflation_factor() function from statsmodel
vif = [variance_inflation_factor(X[x_list].values, i) for i in range(0,len(x_list))]

In [ ]:
#vif values with variable names
#No multicollinearity
for i in range(0,len(x_list)):
    print(x_list[i],":",vif[i])

Although, the vif of const is too high, let's ignore that, since, it is mannually added